In [3]:
directory= "kfold"
domainList = ["art_painting","cartoon","photo","sketch"]
all_data_path=[]
for domain_name in domainList:
    with open(f'{domain_name}_to_label.txt') as f:
        lines = f.readlines()
        for line in lines:
            all_data_path.append( str("kfold"+line.split()[0]))

In [5]:
len(all_data_path)

400

In [6]:
import json
f = open('labelFile.json')
data = json.load(f)



for each group find uniqe discriptions and usethem to label are files

In [8]:
artpaintDescriptions=[]
cartoonDescriptions=[]
photoDescriptions=[]
sketchDescriptions=[]
for info in data:
    if str(info["image_name"]).startswith("art_painting"):
        artpaintDescriptions.append(info["descriptions"])
    elif str(info["image_name"]).startswith("cartoon"):
        cartoonDescriptions.append(info["descriptions"])
    elif str(info["image_name"]).startswith("photo"):
        photoDescriptions.append(info["descriptions"])
    else:
        sketchDescriptions.append(info["descriptions"])



In [12]:
import pandas as pd
columns = ['Level of details','Edges','Color saturation',
           'Color shades','Background','Single instance',
           'Text','Texture','Perspective']
artPaintingDf = pd.DataFrame(artpaintDescriptions, columns=columns)
photoDf = pd.DataFrame(photoDescriptions, columns=columns)
cartoonDf = pd.DataFrame(cartoonDescriptions, columns=columns)
sketchDf = pd.DataFrame(sketchDescriptions, columns=columns)

In [14]:
uniqeValues = artPaintingDf["Level of details"].unique()

array(['detailed, digital, delineated, artificial', 'detailed, intricate',
       'detailed, grainy', 'delineated, grainy level details details',
       'digital, detailed', 'mid level details',
       'detailed, artificial, digital',
       'detailed, representational, grainy',
       'grainy, detailed, representational', 'detailed, delineated',
       'grainy, representational level details details',
       'detailed, elaborate', 'high level details',
       'detailed, representational', 'detailed, grainy, elaborate',
       'low level details', 'delineated, patchy level details details'],
      dtype=object)